In [1]:
using AutoRisk
using AutoViz
using CommandLineFlags
using Discretizers
using PGFPlots
using Colors
include("../collection/collect_heuristic_dataset.jl")
include("../collection/heuristic_dataset_config.jl")
include("../scene_gen/fit_bayes_net.jl");

In [2]:
# load flags from an existing dataset
# these flags should be the ones ultimately used in evaluation
dataset_filepath = "../../data/datasets/may/bn_aug_5_sec_10_timestep_2.h5"
flags = h5readattr(dataset_filepath, "risk")
fixup_types!(flags)
# only collect a single timestep
flags["feature_timesteps"] = 1
# NOTE: this value should be set manually
# and determines for how many runs each scene should be simulated
flags["num_monte_carlo_runs"] = 1
flags["extract_behavioral"] = true

# debug
flags["num_lanes"] = 1
flags["sampling_time"] = 5.

col = build_dataset_collector("", flags);

In [3]:
# selects relevant features and adds to data
function extract_bn_features( 
        features::Array{Float64}, 
        feature_names::Array{String},
        index::Int
    )
    bn_features = zeros(7)
    bn_features[1:5] = extract_base_features(features, feature_names)[:,index]
    bn_features[6] = extract_aggressiveness(features, feature_names, 
        rand_aggressiveness_if_unavailable = false)[index]
    bn_features[7] = extract_is_attentive(features, feature_names, 
        rand_attentiveness_if_unavailable = false)[index] 
    return bn_features
end

extract_bn_features (generic function with 1 method)

In [4]:
function get_prior_data(bn::BayesNet, n_samples::Int = 1000)
    return rand(bn, n_samples)
end

get_prior_data (generic function with 2 methods)

In [5]:
function cem(
            col::DatasetCollector, 
            y::Float64;
            max_iters::Int = 10,
            N::Int = 1000, 
            percentile::Float64 = .50, 
            target_indices::Vector{Int} = [1,2,3,4,5]
        )
    # initialize
    n_vars = length(keys(col.gen.base_bn.name_to_index))
    n_targets, n_vehicles = size(col.eval.targets)
    var_edges = col.gen.assignment_sampler.var_edges
    discs = get_discretizers(var_edges)
    top_k = Int(ceil(percentile * N))
    proposal_vehicle_index = get_target_vehicle_index(col.gen, col.roadway)
    
    # allocate containers
    utilities = zeros(N)
    weights = zeros(N)
    data = zeros(n_vars, N)
    for iter in 1:max_iters
        println("iter: $(iter)\ty_hat: $(mean(utilities))")
        
        # reset
        fill!(utilities, 0.)
        fill!(weights, 0.)
        
        # generate and evaluate a scene from the current bayes net
        for scene_idx in 1:N
            
            # set seed
            seed = iter * N + scene_idx
            
            # sample a scene + models
            rand!(col, seed)
            
            # evaluate this scene
            evaluate!(col.eval, col.scene, col.models, col.roadway, seed)
            
            # extract the relevant information
            utilities[scene_idx] = mean(col.eval.targets[target_indices, proposal_vehicle_index])
            weights[scene_idx] = col.gen.weights[proposal_vehicle_index]
            data[:, scene_idx] = extract_bn_features(col.eval.features[:,end,:], feature_names(col.eval.ext), proposal_vehicle_index)
        end
                
        # select top percentile of population
        indices = sortperm(utilities .* weights)[top_k:end]

        # refit bayesnet
        disc_features = discretize_features(data, discs)
        training_data = form_training_data(disc_features)
        training_data = vcat(training_data, get_prior_data(col.gen.base_bn))
        col.gen.prop_bn = fit_bn(training_data)
        
        # check if the target probability has been sufficiently optimized
        if mean(utilities) > y
            break
        end
    end
    return col.gen.prop_bn
end

cem (generic function with 1 method)

In [ ]:
cem(col, .5, max_iters = 10, N = 100)

iter: 1	y_hat: 0.0
